<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStardust/blob/main/_Recursive_circle_pattern_with_the_gradient_background.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install noise

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from noise import pnoise2
import random
from PIL import Image, ImageDraw, ImageFilter

# Recursive function to draw circles with randomness and noise
def draw_circle(ax, center, radius, depth, color_map):
    if depth > 0:
        # Randomly alter the color and circle radius with Perlin noise
        color = color_map[depth % len(color_map)]
        noisy_radius = radius * (0.8 + 0.2 * pnoise2(center[0] * 0.5, center[1] * 0.5, octaves=2))
        circle = plt.Circle(center, noisy_radius, color=color, fill=False, linewidth=1.5)
        ax.add_artist(circle)

        # Recursive calls with modified positions and radius for randomness
        for angle in np.linspace(0, 2 * np.pi, num=5, endpoint=False):  # More angles for added complexity
            new_radius = radius * (0.4 + 0.2 * random.random())
            x = center[0] + noisy_radius * np.cos(angle)
            y = center[1] + noisy_radius * np.sin(angle)
            draw_circle(ax, (x, y), new_radius, depth - 1, color_map)

# Generate gradient background
def gradient_background(width, height, start_color, end_color):
    image = Image.new("RGB", (width, height), start_color)
    draw = ImageDraw.Draw(image)

    for i in range(height):
        r = start_color[0] + (end_color[0] - start_color[0]) * i // height
        g = start_color[1] + (end_color[1] - start_color[1]) * i // height
        b = start_color[2] + (end_color[2] - start_color[2]) * i // height
        draw.line((0, i, width, i), fill=(r, g, b))

    return image

# Convert matplotlib figure to PIL Image
def fig_to_image(fig):
    fig.canvas.draw()
    image = np.array(fig.canvas.renderer.buffer_rgba())
    return Image.fromarray(image)

# Parameters for the artwork
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_aspect('equal')
ax.axis('off')
color_map = ["#e74c3c", "#3498db", "#9b59b6", "#1abc9c", "#f1c40f", "#34495e"]

# Start recursive drawing with randomness
draw_circle(ax, (0, 0), 1, 5, color_map)

# Apply blur to the figure image
fig_image = fig_to_image(fig).filter(ImageFilter.GaussianBlur(radius=2))

# Generate the gradient background
bg_image = gradient_background(800, 800, (30, 30, 30), (200, 200, 255))

# Overlay the figure image onto the gradient background
final_image = Image.alpha_composite(bg_image.convert("RGBA"), fig_image)
final_image.show()